In [ ]:
# default_exp main

In [ ]:
# export

import argparse
import logging

from cococleaner.json_file import *
from cococleaner.json_tree import *
from cococleaner.crop_tree import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [ ]:
# export 

def get_parser():
    parser = argparse.ArgumentParser(
        description="Tool for converting datasets in COCO format between different formats"
    )
    subparsers = parser.add_subparsers(help='mode')
    parser_convert = subparsers.add_parser('a', help='a help')

    parser.add_argument("--in_json_path", type=Path, required=True)
    parser.add_argument("--in_crop_tree_path", type=Path)
    parser.add_argument("--out_path", type=Path, required=True)
    parser.add_argument("--out_format", choice=['json_file', 'json_tree', 'crop_tree'], required=True)
    parser.add_argument("--overwrite", action='store_true')
    return parser


In [ ]:
# export

def main(args):
    logger.info(f'Arguments: {args}')
    in_json_path = args.in_json_path
    in_crop_tree_path = args.in_crop_tree_path
    
    out_path = args.out_path
    out_format = args.out_format
    overwrite = args.overwrite
    
    if in_crop_tree and out_format == 'crop_tree':
        raise ValueError('Incompatible options: --in_crop_tree=... '
                         'and --out_format=crop_tree')
    
    coco = None
    if in_json_path.is_file():
        ext = in_json_path.suffix
        if ext != '.json':
            raise ValueError(f'Expect .json file as input, got: {in_json_path}')
        coco = load_json_file(in_json_path)
    elif in_json_path.is_dir():
        coco = load_json_tree(in_json_tree)
    
    if coco is None:
        raise ValueError(f'Neither json file nor json tree found in path: {in_json_path}')
    
    if in_crop_tree:
        coco = load_crop_tree(in_crop_tree_path, coco)
    
    if out_format == 'json_file':
        dump_fun = dump_json_file
    elif out_format == 'json_tree':
        dump_fun = dump_json_tree
    elif out_format == 'crop_tree':
        dump_fun = dump_crop_tree
    else:
        raise ValueError(out_format)
    dump_fun(coco, out_path, skip_nulls=True, overwrite=overwrite)
    
    logger.info(f'[+] Success: see {out_path}: {list(out_path.iterdir())}')